In [1]:
#Bowel Movement Frequency
#Clinical Labs Analysis
#Author: James P. Johnson
#Credit to Christian Diener, PhD as noted
#v 3-7-23

# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns

# Get the library
from arivale_data_interface import *
import arivale_data_interface as adi

# Set display options
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.display import display

# Apply these settings so the library knows where the data is
adi.apply_isb_config()

# If you want plots to show up in the notebook you need to run this
%matplotlib inline
plt.rcParams["figure.figsize"] = [10, 10]

In [2]:
#Obtain Bowel Movement Frequency (BMF) data:
bowel = adi.get_snapshot('assessments')
bowel = bowel.sort_values('days_in_program',ascending=True) # Sort df by days first
bowel = bowel.drop(columns = ['vendor','days_in_program','days_since_first_call','days_since_first_draw']) # Drop days and vendor, unneeded from this point
bowel = bowel.drop_duplicates(subset='public_client_id') # Keep only the first timepoint for each public_client_id
bowel = bowel.fillna(0) # Fill in empty BMF data with zero
bowel = bowel.set_index('public_client_id') # Arrange df by index of PCI
bowel = bowel[['assessment:digestion:bowel-movements:enum']]
bowel = bowel.rename(columns={"assessment:digestion:bowel-movements:enum":"bowel"}) # rename column to "bowel"
bowel = bowel[bowel['bowel'] != 0] #remove participants with no bowel data, reduce from 5764 -> 3955 individuals
bowel_list = ['(1) 2 or fewer times per week', '(2) 3-6 times per week', '(3) 1-3 times daily', '(4) 4+ times daily']
bowel_r = [1,2,3,4] #convert bowel responses to ordinal bowel movement frequency categories
bowel.bowel = bowel.bowel.replace(to_replace = bowel_list, value = bowel_r)
# 3955 individuals
bowel

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:101: FutureWarning: The default value of regex will change from True to False in a future version.
  t = pandas.to_datetime(x.str.replace("^arivale_snapshot.*(\\d{4}-\\d{2}-\\d{2}_\\d{4})$", "\\1"), format="%Y-%m-%d_%H%M")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:202: FutureWarning: The default value of regex will change from True to False in a future version.
  colnames = hdr.loc[~hdr.str.contains("^#")].head(1).str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
  hdr = hdr.loc[hdr.str.contains("^#")].str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_in

,bowel
public_client_id,
01092042,3
01568974,3
01684342,2
01601230,3
01124029,3
...,...
01395286,3
01232614,3
01181682,3


In [3]:
#Get the clinical data:
chem=adi.get_snapshot('chemistries')
chemdf = chem.sort_values(by='days_in_program') #keep only first date data
chemdf = chemdf.drop_duplicates(subset='public_client_id')
chemdf = chemdf.set_index('public_client_id')
chemdf = chemdf.iloc[:,12:] #keep only relevant columns
chem_nonnancounts=(~chemdf.isna()).sum() # sum up times any columns have non-NA values
chemdf = chemdf.loc[:, chem_nonnancounts/chemdf.shape[0] >= .70] #this sum of zero values/num of total rows should less than 30% to proceed with the cohort
chemdf = chemdf.fillna(chemdf.median())
chemdf = chemdf.dropna()
chemdf = chemdf.reset_index()
chemdf = chemdf.set_index('public_client_id')
chemdf #4881 individuals

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:101: FutureWarning: The default value of regex will change from True to False in a future version.
  t = pandas.to_datetime(x.str.replace("^arivale_snapshot.*(\\d{4}-\\d{2}-\\d{2}_\\d{4})$", "\\1"), format="%Y-%m-%d_%H%M")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:202: FutureWarning: The default value of regex will change from True to False in a future version.
  colnames = hdr.loc[~hdr.str.contains("^#")].head(1).str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
  hdr = hdr.loc[hdr.str.contains("^#")].str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_in

,"ADIPONECTIN, SERUM",ALAT (SGPT),ALBUMIN,ALKALINE PHOSPHATASE,ARACHIDONIC ACID,ASAT (SGOT),BASOPHILS,BASOPHILS ABSOLUTE,"BILIRUBIN, TOTAL",BUN/CREAT RATIO,CALCIUM,CARBON DIOXIDE (CO2),CHLORIDE,"CHOLESTEROL, TOTAL","CREATININE ENZ, SER",CRP HIGH SENSITIVITY,DHA,DPA,EOSINOPHILS,EOSINOPHILS ABSOLUTE,EPA,FERRITIN,"GFR, MDRD","GFR, MDRD, AFRICAN AM",GGT,GLOBULIN,GLUCOSE,GLYCOHEMOGLOBIN A1C,HDL CHOL DIRECT,HDL PARTICLE NUMBER,HEMATOCRIT,HEMOGLOBIN,HOMA-IR,"HOMOCYSTEINE, SERUM",IMMATURE GRANULOCYTES,IMMATURE GRANULOCYTES ABSOLUTE,INSULIN,LDL PARTICLE NUMBER,LDL SMALL,LDL-CHOL CALCULATION,LDL_SIZE,LINOLEIC_ACID,LPIR_SCORE,LYMPHOCYTES,LYMPHOCYTES ABSOLUTE,"MAGNESIUM, SERUM",MCH,MCHC,MCV,"MERCURY, BLOOD",METHYLMALONIC ACID,MONOCYTES,MONOCYTES ABSOLUTE,OMEGA-3 INDEX,OMEGA-6/OMEGA-3 RATIO,OMEGA_3_TOTAL,OMEGA_6_TOTAL,PLATELET COUNT THOUSAND,POTASSIUM,"PROTEIN, TOTAL SERUM",RDW,RED CELL COUNT,SODIUM,TOTAL NEUTROPHILS,TOTAL NEUTROPHILS AB,TRIGLYCERIDES,Triglyceride HDL Ratio,UREA NITROGEN,URIC ACID,"VITAMIN D, 25-OH TOT",WHITE CELL COUNT
public_client_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01074064,9.3,11.0,4.1,41.0,10.7,17.0,1.0,0.0,0.2,17.0,9.0,24.0,101.0,165.0,0.76,2.74,3.1,0.8,3.0,0.1,0.7,27.0,103.0,118.0,17.0,2.1,90.0,5.3,69.0,37.80,42.2,13.8,1.111111,11.4,0.0,0.0,5.0,965.0,328.0,81.0,20.9,24.2,25.0,44.0,1.8,1.9,30.7,32.7,94.0,1.6,163.0,7.0,0.3,4.6,8.5,4.6,38.9,229.0,4.5,6.2,13.0,4.49,138.0,45.0,1.8,75.0,1.086957,13.0,4.7,44.7,3.9
01920535,19.4,12.0,4.0,68.0,10.9,17.0,0.6,0.0,0.2,14.0,9.1,23.0,104.0,225.0,0.86,1.04,2.5,1.1,2.0,0.1,0.5,162.0,75.0,87.0,10.0,2.5,95.0,5.5,61.0,30.00,42.1,14.0,1.172840,11.4,0.0,0.0,5.0,1217.0,132.0,147.0,22.1,29.7,25.0,32.8,1.7,2.2,30.3,33.4,91.0,2.2,166.0,7.0,0.4,4.1,10.7,4.1,43.7,239.0,4.3,6.5,13.6,4.65,141.0,57.0,3.0,86.0,1.409836,12.0,5.5,17.8,5.3
01954001,2.3,25.0,4.3,92.0,7.7,19.0,0.0,0.0,0.5,26.0,9.5,23.0,101.0,176.0,0.46,2.90,2.3,0.9,1.0,0.0,0.7,244.0,110.0,127.0,21.0,2.5,132.0,5.9,38.0,34.90,37.8,13.1,8.767407,6.2,0.0,0.0,26.9,1195.0,899.0,74.0,19.6,25.8,72.0,46.0,2.5,2.1,32.3,34.7,93.0,4.0,88.0,7.0,0.4,3.9,9.3,3.9,36.2,295.0,4.2,6.8,12.5,4.06,140.0,46.0,2.5,322.0,8.473684,12.0,5.3,13.5,5.5
01005100,8.6,16.0,4.0,49.0,12.5,53.0,0.0,0.0,0.5,23.0,9.0,21.0,103.0,185.0,0.69,0.78,3.1,1.3,2.0,0.1,1.0,24.0,103.0,119.0,13.0,2.1,95.0,5.4,74.0,31.70,38.0,12.7,2.204938,7.4,0.0,0.0,9.4,912.0,90.0,99.0,21.2,23.8,32.0,26.0,1.2,2.2,29.5,33.4,88.0,2.9,144.0,6.0,0.3,5.4,7.3,5.4,39.3,229.0,4.4,6.1,13.2,4.31,139.0,66.0,3.0,62.0,0.837838,16.0,4.8,47.7,4.5
01627898,7.2,21.0,4.6,51.0,11.8,20.0,0.6,0.0,2.0,14.0,9.3,24.0,104.0,194.0,0.92,2.84,3.3,1.5,2.0,0.1,1.4,122.0,98.0,113.0,14.0,2.3,85.0,5.1,59.0,34.35,42.1,14.0,0.755556,12.1,0.0,0.0,3.6,1230.0,349.0,111.0,21.2,25.6,32.0,32.8,1.7,2.0,30.3,33.4,91.0,2.0,144.0,7.0,0.4,6.2,6.6,6.2,40.8,239.0,4.0,6.9,13.6,4.65,143.0,57.0,3.0,91.0,1.534483,13.0,7.1,51.8,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
01034328,30.7,46.0,3.6,205.0,7.5,86.0,0.0,0.0,0.5,19.0,9.2,28.0,101.0,179.0,0.64,3.24,3.2,0.9,3.0,0.1,0.7,102.0,96.0,111.0,309.0,2.4,84.0,4.9,85.0,24.90,34.3,11.5,1.576296,11.6,0.0,0.0,7.6,650.0,90.0,80.0,22.1,21.3,44.0,13.0,0.5,1.8,37.8,33.5,113.0,8.8,118.0,10.0,0.3,4.8,6.6,4.8,31.5,150.0,4.2,6.0,14.0,3.04,141.0,74.0,2.5,72.0,0.847059,12.0,6.7,25.8,3.5
QIRL,16.3,15.0,4.1,77.0,9.6,23.0,1.0,0.0,0.3,17.0,9.0,20.0,99.0,184.0,0.64,4.20,3.3,1.6,2.0,0.1,1.8,41.0,108.0,125.0,35.0,2.6,96.0,5.2,88.0,37.50,40.7,13.7,1.161481,8.4,0.0,0.0,4.9,684.0,90.0,83.0,21.2,25.2,25.0,25.0,1.2,2.0,28.7,33.7,85.0,0.0,137.0,7.0,0.4,6.7,5.7,6.7,38.3,179.0,4.4,6.7,15.2,4.77,137.0,65.0,3.1,65.0,0.738636,11.0,3.5,33.3,4.9
HX842645,7.2,30.0,4.7,49.0,8.3,33.0,0.6,0.0,0.6,16.0,9.2,23.0,100.0,272.0,0.99,0.72,2.1,1.1,2.0,0.1,0.7,110.0,96

In [4]:
#Get BMI data
bmi = adi.get_snapshot('weight_monthly')
bmi = bmi[['public_client_id','days_in_program','BMI_CALC']] # Keep relevant columns
bmi = bmi.sort_values('days_in_program',ascending=True) # Sort by days in program
bmi = bmi.sort_values('public_client_id',ascending=True) # Sort by PCI
bmi = bmi.drop('days_in_program', axis=1) # Drop days column (unneeded at this point)
bmi = bmi.drop_duplicates('public_client_id') # Drop duplicate PCI
bmi = bmi.set_index('public_client_id') # Arrange index by PCI
bmi = bmi.dropna() # Drop PCI with no BMI data
bmi #5567 individuals

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:101: FutureWarning: The default value of regex will change from True to False in a future version.
  t = pandas.to_datetime(x.str.replace("^arivale_snapshot.*(\\d{4}-\\d{2}-\\d{2}_\\d{4})$", "\\1"), format="%Y-%m-%d_%H%M")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:202: FutureWarning: The default value of regex will change from True to False in a future version.
  colnames = hdr.loc[~hdr.str.contains("^#")].head(1).str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
  hdr = hdr.loc[hdr.str.contains("^#")].str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_in

,BMI_CALC
public_client_id,
01000261,22.594123
01000552,31.619378
01001181,39.047808
01001298,25.948604
01001548,36.119222
...,...
QIRL,31.942410
ZU273983,32.042139
ZU612255,36.045023


In [5]:
#Obtain "eGFR" calculated GFR data (credit: Alexandra Ralevski) code chunk below:
################################################################################################
chems = adi.get_snapshot('chemistries', clean=True)
cl = adi.get_snapshot('clients', clean=True)
chems_dems = join_clients(cl, chems)
#these are the relevant column names
[col for col in chems.columns if 'CREAT' in col]
[col for col in chems.columns if 'GFR' in col]
def f(sex, age, creatinine): 
    if sex == 'F':
        if (creatinine/0.7) <=1:
            return 142*(((creatinine/0.7)**-0.241) * (0.9938**age) * 1.012)
        else:
            return 142*(((creatinine/0.7)**-1.200) * (0.9938**age) * 1.012)
    if sex == 'M':
        if (creatinine/0.9) <=1:
            return 142*(((creatinine/0.9)**-0.302) * (0.9938**age))
        else:
            return 142*(((creatinine/0.7)**-1.200) * (0.9938**age))

creat_calc = chems_dems[['public_client_id','sex', 'age', 'CREATININE_ENZ__SER']]

creat_calc['eGFR'] = creat_calc.apply(lambda x: f(x.sex, x.age, x.CREATININE_ENZ__SER), axis=1)
#creat_calc.to_csv('eGFR.csv',index = False)
################################################################################################

#prepare eGFR dataframe:
eGFR = creat_calc
#eGFR = eGFR.set_index('public_client_id')
eGFR = eGFR[['public_client_id','eGFR']] #select relevant columns
#drop duplicate entries to reduce 12316 entries across all individuals
#to 6133 individuals
eGFR = eGFR.drop_duplicates('public_client_id')
eGFR = eGFR.dropna()
eGFR = eGFR.set_index('public_client_id')
eGFR

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:101: FutureWarning: The default value of regex will change from True to False in a future version.
  t = pandas.to_datetime(x.str.replace("^arivale_snapshot.*(\\d{4}-\\d{2}-\\d{2}_\\d{4})$", "\\1"), format="%Y-%m-%d_%H%M")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:202: FutureWarning: The default value of regex will change from True to False in a future version.
  colnames = hdr.loc[~hdr.str.contains("^#")].head(1).str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
  hdr = hdr.loc[hdr.str.contains("^#")].str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_in

,eGFR
public_client_id,
01001298,99.024207
01001467,100.959522
01001548,105.411124
01001621,69.439409
01001661,120.947576
...,...
QIRL,111.687296
ZU273983,118.179626
ZU612255,70.256126


In [6]:
#Now, get the age data from the "assessments" snapshot
bio=adi.get_snapshot('clients')
a = bio['sex']
b = bio['age']
c = bio['public_client_id']
biodf = pd.DataFrame({'sex':a, 'age':b, 'public_client_id':c}) # Get df with desired columns
biodf = biodf[['sex','age','public_client_id']]
biodf = biodf.set_index('public_client_id') # Arrange index by PCI
biodf = biodf.replace(to_replace = 'F', value = 'W') # Replace F with W
biodf = biodf.dropna()
biodf #6133 individuals

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:101: FutureWarning: The default value of regex will change from True to False in a future version.
  t = pandas.to_datetime(x.str.replace("^arivale_snapshot.*(\\d{4}-\\d{2}-\\d{2}_\\d{4})$", "\\1"), format="%Y-%m-%d_%H%M")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:202: FutureWarning: The default value of regex will change from True to False in a future version.
  colnames = hdr.loc[~hdr.str.contains("^#")].head(1).str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
  hdr = hdr.loc[hdr.str.contains("^#")].str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_in

,sex,age
public_client_id,,
01000552,M,54.0
01000978,W,52.0
01001181,W,48.0
01001298,W,61.0
01001467,W,31.0
...,...,...
QIRL,W,44.0
ZU273983,W,32.0
ZU612255,W,35.0


In [7]:
# Merge dfs, adding clinical labs, bmi, age, and sex data
clinic = chemdf
clinic = pd.merge(bowel, clinic, left_index = True, right_index = True)
clinic = pd.merge(eGFR, clinic, left_index = True, right_index = True)
clinic = pd.merge(bmi, clinic, left_index = True, right_index = True)
clinic = pd.merge(biodf, clinic, left_index = True, right_index = True)
clinic

,sex,age,BMI_CALC,eGFR,bowel,"ADIPONECTIN, SERUM",ALAT (SGPT),ALBUMIN,ALKALINE PHOSPHATASE,ARACHIDONIC ACID,ASAT (SGOT),BASOPHILS,BASOPHILS ABSOLUTE,"BILIRUBIN, TOTAL",BUN/CREAT RATIO,CALCIUM,CARBON DIOXIDE (CO2),CHLORIDE,"CHOLESTEROL, TOTAL","CREATININE ENZ, SER",CRP HIGH SENSITIVITY,DHA,DPA,EOSINOPHILS,EOSINOPHILS ABSOLUTE,EPA,FERRITIN,"GFR, MDRD","GFR, MDRD, AFRICAN AM",GGT,GLOBULIN,GLUCOSE,GLYCOHEMOGLOBIN A1C,HDL CHOL DIRECT,HDL PARTICLE NUMBER,HEMATOCRIT,HEMOGLOBIN,HOMA-IR,"HOMOCYSTEINE, SERUM",IMMATURE GRANULOCYTES,IMMATURE GRANULOCYTES ABSOLUTE,INSULIN,LDL PARTICLE NUMBER,LDL SMALL,LDL-CHOL CALCULATION,LDL_SIZE,LINOLEIC_ACID,LPIR_SCORE,LYMPHOCYTES,LYMPHOCYTES ABSOLUTE,"MAGNESIUM, SERUM",MCH,MCHC,MCV,"MERCURY, BLOOD",METHYLMALONIC ACID,MONOCYTES,MONOCYTES ABSOLUTE,OMEGA-3 INDEX,OMEGA-6/OMEGA-3 RATIO,OMEGA_3_TOTAL,OMEGA_6_TOTAL,PLATELET COUNT THOUSAND,POTASSIUM,"PROTEIN, TOTAL SERUM",RDW,RED CELL COUNT,SODIUM,TOTAL NEUTROPHILS,TOTAL NEUTROPHILS AB,TRIGLYCERIDES,Triglyceride HDL Ratio,UREA NITROGEN,URIC ACID,"VITAMIN D, 25-OH TOT",WHITE CELL COUNT
public_client_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01001298,W,61.0,25.948604,99.024207,3,7.2,25.0,4.2,63.0,11.7,28.0,0.6,0.0,0.3,18.0,9.0,24.0,101.0,271.0,0.68,0.60,3.0,1.2,2.0,0.1,1.2,57.0,95.0,109.0,15.0,2.4,94.0,5.6,64.0,35.70,42.1,14.0,1.601481,7.5,0.0,0.0,6.9,2307.0,893.0,187.0,21.5,25.0,25.0,32.8,1.7,2.0,30.3,33.4,91.0,2.0,144.0,7.0,0.4,5.4,7.4,5.4,39.7,239.0,4.1,6.6,13.6,4.65,140.0,57.0,3.0,100.0,1.562500,12.0,4.1,41.7,5.3
01001548,M,49.0,36.119222,105.411124,3,7.2,27.0,4.5,57.0,10.1,21.0,0.6,0.0,0.6,18.0,9.4,25.0,102.0,205.0,0.88,1.40,2.0,1.1,2.0,0.1,0.7,173.0,101.0,117.0,26.0,2.4,106.0,5.8,45.0,34.35,42.1,14.0,3.507160,12.5,0.0,0.0,13.4,1230.0,349.0,135.0,21.2,25.1,32.0,32.8,1.7,2.0,30.3,33.4,91.0,2.0,144.0,7.0,0.4,3.8,10.3,3.8,39.1,239.0,4.1,6.9,13.6,4.65,140.0,57.0,3.0,123.0,2.733333,16.0,6.5,25.6,5.3
01001621,W,54.0,23.627456,69.439409,2,11.8,6.0,3.9,67.0,11.2,9.0,1.0,0.1,0.2,20.0,9.2,24.0,102.0,189.0,0.97,1.92,2.2,1.3,3.0,0.2,0.6,68.0,66.0,77.0,8.0,2.2,87.0,5.6,62.0,32.00,40.1,13.7,0.859259,11.6,0.0,0.0,4.0,1392.0,343.0,114.0,21.6,29.6,25.0,28.0,1.6,1.8,29.7,34.2,87.0,0.0,91.0,8.0,0.5,4.1,10.8,4.1,44.2,217.0,4.0,6.1,13.4,4.62,142.0,60.0,3.4,63.0,1.016129,19.0,6.1,30.5,5.8
01001661,W,30.0,21.031478,120.947576,3,12.2,10.0,4.6,36.0,10.8,21.0,1.0,0.0,0.7,15.0,9.3,24.0,100.0,187.0,0.66,1.08,3.0,0.7,2.0,0.1,1.0,66.0,119.0,137.0,11.0,2.4,89.0,5.6,87.0,44.00,39.4,13.4,2.439259,8.3,0.0,0.0,11.1,765.0,90.0,84.0,21.2,27.6,25.0,27.0,1.4,2.2,31.0,34.0,91.0,3.3,95.0,5.0,0.2,4.7,9.0,4.7,42.1,241.0,4.2,7.0,13.2,4.32,140.0,65.0,3.4,78.0,0.896552,10.0,4.2,19.0,5.2
01001798,W,55.0,29.579491,93.963151,1,7.2,26.0,4.5,99.0,12.6,27.0,0.6,0.0,0.4,21.0,9.8,23.0,101.0,183.0,0.75,0.85,2.4,0.9,2.0,0.1,0.3,18.0,91.0,105.0,20.0,2.2,95.0,5.6,57.0,41.30,42.1,14.0,3.800000,8.5,0.0,0.0,16.2,1293.0,694.0,103.0,20.9,28.7,64.0,32.8,1.7,2.0,30.3,33.4,91.0,2.0,144.0,7.0,0.4,3.6,12.4,3.6,44.5,239.0,4.2,6.7,13.6,4.65,140.0,57.0,3.0,116.0,2.035088,16.0,5.9,58.9,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HX981293,M,45.0,24.806553,107.698203,3,6.7,23.0,4.8,59.0,11.6,24.0,0.0,0.0,0.9,19.0,10.0,28.0,101.0,206.0,0.89,3.46,3.9,1.4,1.0,0.0,1.6,76.0,103.0,119.0,46.0,2.8,106.0,5.6,65.0,41.80,41.9,14.1,1.387160,9.2,0.0,0.0,5.3,1457.0,750.0,126.0,21.2,24.9,33.0,32.0,1.8,2.2,29.7,33.7,88.0,9.9,95.0,8.0,0.4,6.9,5.7,6.9,39.3,295.0,4.5,7.6,14.1,4.74,141.0,59.0,3.2,75.0,1.153846,17.0,5.5,31.4,5.5
ZU273983,W,32.0,32.042139,118.179626,2,7.2,13.0,4.4,113.0,9.7,20.0,0.6,0.0,0.4,14.0,9.1,20.0,102.0,133.0,0.69,11.92,1.3,1.1,2.0,0.1,0.6,63.0,116.0,133.0,17.0,2.9,81.0,5.0,37.0,29.60,42.1,14.0,2.260000,8.8,0.0,0.0,11.3,634.0,26

In [8]:
#Grabbing Metadata:
clinic.iloc[:,:5].to_csv('labs_metadata.csv')

#count data
clinic.iloc[:,5:].transpose().to_csv('labs.csv')

#full data
clinic.to_csv('labs_full.csv')

In [9]:
clinic.iloc[:,5:].transpose()

public_client_id,01001298,01001548,01001621,01001661,01001798,01002192,01002471,01002885,01003411,01003555,01003662,01003757,01003758,01004037,01004159,01004350,01004551,01004575,01004596,01004750,01005100,01005890,01005935,01006225,01006695,01007032,01007074,01007520,01008097,01008265,01008415,01008612,01008631,01009390,01009735,01010189,01010305,01010353,01010719,01010869,01011465,01011574,01011640,01012428,01012516,01012545,01012626,01012799,01013166,01013486,01013541,01013762,01013764,01014138,01014546,01014792,01014816,01015440,01015901,01016929,01017180,01017741,01017775,01017820,01018541,01018568,01018937,01019058,01019071,01019279,01019309,01019674,01019750,01020036,01020175,01020229,01020425,01020750,01021265,01021394,01021397,01021487,01021722,01021750,01022339,01022402,01022407,01022858,01023349,01023413,01024105,01024256,01024456,01024630,01025234,01025263,01025634,01025972,01026607,01026921,...,01986408,01986641,01987388,01987648,01987914,01988559,01988577,01988617,01988631,01988867,01989393,01989483,01989581,01989730,01989933,01990047,01990311,01990810,01991181,01992009,01992522,01993490,01993643,01993866,01994088,01994108,01994373,01994483,01994513,01995018,01995109,01995589,01995656,01996666,01997414,01997714,01997759,01997826,01997940,01998370,01998451,01998595,01998890,01998999,01999296,01999351,01999890,01999901,HX002381,HX007614,HX010060,HX025494,HX048925,HX053777,HX098685,HX120133,HX122909,HX123670,HX158083,HX169155,HX177299,HX188953,HX248934,HX314057,HX314213,HX344502,HX346282,HX348624,HX360310,HX409129,HX411182,HX411235,HX447557,HX460562,HX534474,HX548151,HX582798,HX611763,HX662917,HX670773,HX672486,HX694416,HX748976,HX768417,HX770635,HX794171,HX824177,HX851881,HX858710,HX864598,HX884201,HX927178,HX935045,HX940258,HX980884,HX981293,ZU273983,ZU612255,ZU621944,ZU687756
"ADIPONECTIN, SERUM",7.200000,7.200000,11.800000,12.200000,7.200000,6.000000,10.900000,5.300000,12.700000,5.200000,4.000000,4.500000,7.500000,2.000000,7.200000,6.30000,26.600000,13.700000,3.400000,7.200000,8.600000,6.400000,6.600000,12.900000,6.200000,9.800000,16.400000,10.100000,4.900000,23.500000,7.200000,7.000000,10.000000,15.500000,6.000000,8.600000,8.300000,6.100000,2.900000,6.500000,7.200000,7.20,6.100000,8.800000,5.600000,12.100000,12.700000,7.200000,7.200000,7.200000,21.300000,8.000000,8.200000,7.200000,7.200000,7.200000,7.500000,3.30000,7.200000,20.500000,6.200000,6.800000,9.60000,4.100000,15.200000,18.100000,6.200000,4.700000,10.900000,7.400000,6.400000,7.200000,7.200000,6.100000,5.800000,4.500000,11.600000,7.200000,1.400000,7.200000,4.100000,8.400000,7.200000,4.600000,7.200000,4.800000,10.600000,4.600000,3.800000,8.800000,12.400000,8.000000,5.700000,13.900000,3.000000,28.700000,7.200000,20.200000,4.200000,4.300000,...,14.300000,7.200000,10.000000,13.900000,7.000000,3.800000,7.200000,10.900000,3.400000,7.200000,3.800000,2.700000,4.700000,7.200000,12.700000,3.900000,10.700000,7.200000,34.400000,7.200000,4.000000,14.500000,5.100000,9.500000,8.600000,8.600000,20.900000,7.600000,7.200000,4.400000,3.100000,4.100000,4.50000,7.200000,4.800000,7.200000,4.00000,2.900000,7.200000,3.600000,5.90000,2.000000,7.20000,7.100000,7.200000,5.600000,7.200000,5.700000,7.200000,7.200000,10.900000,15.000000,7.200000,5.900000,9.200000,7.200000,4.500000,7.900000,7.200000,7.200000,12.800000,11.000000,7.200000,7.200000,12.700000,10.000000,7.200000,7.200000,7.200000,10.800000,14.400000,7.200000,2.100000,7.200000,21.300000,7.200000,9.200000,9.500000,7.200000,20.300000,7.200000,11.300000,6.600000,4.600000,7.200000,4.400000,7.200000,7.200000,3.700000,7.200000,13.300000,7.500000,2.800000,7.200000,7.200000,6.700000,7.200000,3.400000,3.500000,3.400000
ALAT (SGPT),25.000000,27.000000,6.000000,10.000000,26.000000,52.000000,16.000000,30.000000,16.000000,38.000000,36.000000,33.000000,14.000000,18.000000,13.000000,23.00000,17.000000,19.000000,15.000000,16.000000,16.000000,18.000000,16.000000,22.000000,14.000000,16.000000,9.000000,19.000000,20.000000,6.000000,39.000000,1